In [1]:
import numpy as np
import scipy.stats as st
#import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:
from sklearn.metrics import adjusted_rand_score as ars
from sklearn.mixture import BayesianGaussianMixture as GMM_sklearn
from vbnigmm import GaussMixture as GMM_tk2
from vbnigmm import NormalInverseGaussMixture as NIGMM_tk2

In [3]:
def make_data(d, normality, asymmetry, difficulty, population, normality_prec=5, difficulty_prec=5):
    difficulty_prec += d
    m = len(population)

    mu = st.multivariate_normal(np.zeros(d), np.eye(d)).rvs(size=m)
    cov = difficulty ** 2 * st.wishart(difficulty_prec, np.eye(d) / difficulty_prec).rvs(size=m)
    beta = asymmetry * np.array([st.multivariate_normal(np.zeros(d), c).rvs() for c in cov])
    lmd = st.invgauss(1 / normality_prec, 0, normality * normality_prec).rvs(size=m)

    z = np.concatenate([np.full(n, i) for i, n in enumerate(population)])
    y = st.invgauss(1/lmd[z], 0, lmd[z]).rvs()
    x = np.array([st.multivariate_normal(m, s).rvs() for m, s in zip(mu[z]+y[:,None]*beta[z], y[:,None,None] * cov[z])])
    return x, z + 1

In [7]:
import tensorflow as tf
x = tf.ones((100, 10))
data = tf.data.Dataset.from_tensor_slices(x).batch(100).repeat(10).prefetch(3)
for d in data:
    print(id(d))

139685365859776
139685366178032
139685365859776
139685366178032
139685365859776
139685366178032
139685365859776
139685366178032
139685365859776
139685366178032


In [8]:
import tensorflow as tf
x = tf.ones((100, 10))
data = tf.data.Dataset.from_tensors(x).repeat(10).prefetch(3)
for d in data:
    print(id(d))

139685365859424
139685366178032
139685365859424
139685366178032
139685365859424
139685366178032
139685365859424
139685366178032
139685365859424
139685366178032


In [9]:
id(x)

139685366180320

In [4]:
x, y_true = make_data(d=3, normality=0.3, asymmetry=0.5, difficulty=0.2, population=[100] * 10)
#sns.pairplot(data, hue='cluster', markers='.', palette='prism')
#plt.show()

In [5]:
#x = np.array(data[['x','y','z']])
#y_true = np.array(data['cluster'])

In [10]:
x.shape

(1000, 3)

In [6]:
model = GMM_sklearn(50, n_init=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

/home/takekawa/github/vbnigmm/.venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=50 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


CPU times: user 7.65 s, sys: 1.92 s, total: 9.57 s
Wall time: 2.67 s


0.8079365084106165

In [7]:
model = GMM_tk2()
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:MatMul]

InvalidArgumentError: Incompatible shapes: [49] vs. [0] [Op:AddV2]

In [ ]:
model = NIGMM_fixed(progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = NIGMM_tk2(concentration_prior_type='dd', normality_prior_type='gamma', progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = NIGMM_takekawa(normality_prior_type='gamma', progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = NIGMM_takekawa(concentration_prior_type='dd', progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [11]:
model = NIGMM_tk2()
model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:MatMul]